In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np


In [2]:
'''
Unet based model for detecting salt regions in seismic images
Input:
Image 101x101 [0,1]
Transform:
None(for now)
Output:
101x101 matrix with pixel wise confidence values for salt presence

Structure:
DN: A FC layer to create a kernel that applies on the images
UN: A  unet image classifier

'''


'\nUnet based model for detecting salt regions in seismic images\nInput:\nImage 101x101 [0,1]\nTransform:\nNone(for now)\nOutput:\n101x101 matrix with pixel wise confidence values for salt presence\n\nStructure:\nDN: A FC layer to create a kernel that applies on the images\nUN: A  unet image classifier\n\n'

In [28]:
class DN_(nn.Module):
    '''
    @IN
    img: A tensor representing the grayscaled image
    d: The z index of the image
    
    @OUT
    101*101 tensor representing the features found by kernel(3*3) created from depth value
    '''
    def __init__(self):
        super(DN_,self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(4,9,bias=False),
                                   nn.ReLU())
        self.layer2 = nn.Sequential(nn.Linear(9,9,bias=True),
                                   nn.Sigmoid())
        self.conv1 = nn.Conv2d(1,1,kernel_size=3,stride=1,padding=1,bias=False)
        self.non_linear1 = nn.ReLU()
    def forward(self,x,d):
        ker = torch.from_numpy(np.array([d,d**2,d**3,1])).float()
        ker = self.layer1(ker)
        ker = self.layer2(ker).view(1,1,3,3)
        self.conv1.weight = nn.Parameter(ker)
        x = self.non_linear1(self.conv1(x))
        return x

In [30]:
x = torch.randn(10,1,101,101)
y = torch.randn(10,1)
model =DN_()
model(x,45).shape

torch.Size([10, 1, 101, 101])

In [ ]:
class UN_(nn.Module):
    def __init__(self):
        super(UN_,self).__init__()
        